# Import

In [2]:
import re
import pandas        as pd
import numpy         as np

# Load DataSet

In [4]:
df = pd.read_csv('../dataset/list_auto_cleaning.csv', encoding='utf-8', sep=';')

In [5]:
df.head()

,Unnamed: 0,id_ads,manufacturer_by,model,version,km,year,price,seller,endereco,scrapy_datetime
0,0,7fd72920-850d-43ba-a452-cbae1a6398da,land rover,defender,110 SW SE,81000,08-2015,38000,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
1,1,840de0a3-82a9-454c-a725-508e211af7de,land rover,defender,Defender 90 td5 -MOTORE NUOVO-MECCANICA PERFETTA,147000,11-1999,24900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
2,2,6611c967-ab67-4db5-b28f-37b77fcd332f,land rover,defender,110 SW E,79000,08-2013,39900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
3,3,7484ee53-ebc5-4510-8bf9-032e657837cf,land rover,defender,110 SW E,78000,08-2012,39500,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03
4,4,08c56b66-b680-4f5c-b63a-33ce190b65eb,land rover,defender,110 SW SE,52000,08-2014,42600,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03


# Using REGEX langague to clean and capture data-info

In [139]:
df_clean = df.copy()

In [140]:
# "Extracting the postal code from the vehicle's country of origin. 
# Applying Regex for data cleaning."
df_clean['zip_code'] = df_clean['endereco'].apply(lambda x: int(re.search('\d+', x).group(0) if pd.notnull( x ) else x ))

In [141]:
# Model
df_clean['model'] = 'DEFENDER' + ' ' + df_clean['version'].apply(lambda x: ' '.join(re.findall(r'\b(88|90|110|130)\b', x)) if re.findall(r'\d+', x) else None)

In [142]:
# Version Type
df_clean['engine'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'[A-Za-z]+\d+', x)) if re.findall(r'\d+', x) else None)
df_clean['engine_version'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'(AWD|SW|SE|HSE|MHEV|HT)', x)) if re.findall(r'\d+', x) else None)
df_clean['cc_engine'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'\b\d\.\d', x)) if re.findall(r'\d+', x) else None)

# Replacing model version values.
df_clean['version'] = df_clean.apply(lambda row: row['engine'] + ' ' + row['engine_version'] + ' ' + row['cc_engine'] if row['engine'] is not None and row['engine_version'] is not None and row['cc_engine'] is not None else None, axis=1)


In [143]:
#Country
df_clean['country'] = df_clean['endereco'].apply(lambda x: str(re.search(r"([A-Z]{2})", x).group(0) if pd.notnull( x ) else x ))

In [144]:
# DataTime cleaning

df_clean['year'] = df_clean['year'].str.replace('-', '/')


for i in range(len(df_clean)):
    if df_clean['year'][i] == 'new' or df_clean['year'][i] == 'unknown' or df_clean['year'][i] == 'None':
        df_clean.at[i, 'year'] = '01/2023'


df_clean['year'] = pd.to_datetime(df_clean['year'], format='%m/%Y')

df_clean['year'] = df_clean['year'].dt.year


## DataFrame Final

In [162]:
df_final = pd.DataFrame(df_clean[['id_ads', 'manufacturer_by', 'model', 'version', 'km', 'year',
                                  'price', 'seller', 'endereco', 'scrapy_datetime', 'zip_code', 'country']])

In [164]:
df_final.dtypes

id_ads             object
manufacturer_by    object
model              object
version            object
km                 object
year                int32
price               int64
seller             object
endereco           object
scrapy_datetime    object
zip_code            int64
country            object
dtype: object

## Save the final dataset


In [165]:
rows = df_final.shape[0]
print(rows)
df_final.to_csv('../dataset/df_final.csv', encoding='utf-8', sep=';')

399
